In [ ]:
mutable struct TreeNode
  parent::Int             # dans l'arbre, indice du noeud parent
  children::Vector{Int}   # dans l'arbre, indices de noeuds enfants
  ub::Float32             # borne supérieure du noeud
  setzero::Array{Tuple}   # liste des arcs non utilisés (à 0) à cause des règles de branchement
  setone::Array{Tuple}    # liste des arcs obligatoirement utilisés (à 1) à cause des règles de branchement
end

## Filtrer les colonnes par rapport aux contraintes

In [ ]:
function calculate_columns(node)
    node_pool = Array{Dict{String,Any},1}() # pool de colonnes (i.e des cycles) pour ce noeud
    
    # Colonne artificielle     
    node_pool = vcat(node_pool, column_pool[1:KEEP_INIT_COLUMNS])
    
    for c in (KEEP_INIT_COLUMNS+1):size(column_pool,1)
        add = true
        
        # Vérification que la condition de branchement des arcs non-utilisés est vérifiée
        for (i,j) in tree[node].setzero
            if (i,j) in column_pool[c]["oneedges"]
                add = false
                @goto next_cycle
            end
        end
        # Et vérification que la condition de branchement des arcs obligatoirement utilisés est vérifiée
        for (i,j) in tree[node].setone
            if !((i,j) in column_pool[c]["oneedges"])
                add = false
                @goto next_cycle
            end
        end

        if add
            push!(node_pool, column_pool[c])
        end
        @label next_cycle
    end

    return node_pool
end

In [ ]:
# Tranformation d'une solution maître sous forme z en solution sous forme x
function calculate_xsol(mastersol, node_pool)
    x = Dict{Tuple{Int,Int},Float32}()
    
    for e in edges(GRAPH)
        i,j = e.src,e.dst
        x[(i,j)] = sum([ mastersol[c] for c in 1:size(mastersol,1) if (i,j) in node_pool[c]["oneedges"] ])
    end  
    return x
end

In [ ]:
function Process_Node(nodeindex)
    println("\e[93mNOEUD n°$nodeindex :\e[00m")
    global nodestobedeleted = []
    
    # Récupération des colonnes compatibles avec les contraintes de branchement
    node_pool = calculate_columns(nodeindex)
    
    # Création du problème maître restreint
    # (on récupère les références des contraintes et des variables z ainsi que le modèle en lui-même)
    cyclechoice, z, mastermodel = node_master(node_pool)
    
    
    # GÉNÉRATION DE COLONNES
    while true
        
        # RÉSOLUTION DU PROBLÈME MAÎTRE RESTREINT
        optimize!(mastermodel)
        
        # si le maître restreint est infaisable alors ce noeud est élagé
        if termination_status(mastermodel) == MOI.INFEASIBLE 
            println(" \e[93m|\e[31m  Ce noeud est irréalisable et donc élagué \e[00m")
            return []
        end
        
        # Récupération du problème maître restreint
        #  - la valeur optimale :
        value = JuMP.objective_value(mastermodel)
        
        #  - la solution optimale :
        solution = Array{Float32,1}(undef,size(node_pool,1))
        for c in 1:size(node_pool,1)
            solution[c] = JuMP.value.(z)[c]
        end
        solflat = collect(Iterators.flatten(solution))
        
        #  - les solutions duales optimales associées aux contraintes :
        π = -JuMP.dual.(cyclechoice)                                ##### le - ici à cause de la maximisation

        
        # La solution est-elle entière ?
        #   si oui, alors on met à jour la borne primale
        if maximum(solflat - floor.(solflat)) <= ϵ
            # mise à jour avec la meilleure borne primale connue (borne inférieure ici)
            if value > LB                                                   ############# j'ai mis > au lieu de >=
                global LB = value
                println(" \e[93m|\e[32m  Meilleure solution réalisable (entière) de valeur $value trouvée \e[00m")
            end
            
            # et on retirera les noeuds non traitées dont les bornes supérieures ne peuvent pas être meilleure
            # que la borne inférieure du problème entier
            for i in 1:length(Queue)-1                                     
                if tree[Queue[i]].ub <= LB                                   ############### À mettre dans le if ci-dessus ?
                    push!(nodestobedeleted,i)
                end
            end
        end
        
        
        # RÉSOLUTION DU SOUS-PROBLÈME
        # récupération de la valeur objectif du sous-problème et de la colonne associée
        SPobj, column = subproblem(π, nodeindex)
        
        # si le sous-problème n'est pas réalisable alors ce noeud est élagé 
        if SPobj == -Inf
            println(" \e[93m|\e[31m  Ce noeud est irréalisable et donc élagué \e[00m")
            return []
        end
        
        
        # MISE À JOUR DE BORNES        
        nodelb = value                                                         ########## Vient du problème restreint
        nodeub = sum(π) + (nv(GRAPH)/2)*SPobj                                  ########## Vient du Lagrangien dual

                                                                  
        # Si le coût réduit de la colonne est positive alors on l'ajoute
        # (à noter 'SPobj' est le coût réduit ici)
        if SPobj > 0                                                           ##################### strictement ?
            # Ajout de la colonne au pool de toutes les colonnes ainsi qu'au pool de colonnes de ce noeud
            column["cost"] = sum(WEIGHTS[(i,j)] for (i,j) in column["oneedges"])
            push!(column_pool, column)
            push!(node_pool, column)

            # Ajout de la colonne dans le modèle
            #   une variable z est d'abord ajoutée
            push!(z, @variable(mastermodel,lower_bound=0,upper_bound=1))
            set_name(z[end], "z_$(size(node_pool,1))") # nom de la variable

            #   ajout ensuite des coefficients de la variable dans les contraintes de choix de cycles
            for i in VERTICES
                set_normalized_coefficient(cyclechoice[i], z[end], column["vertices"][i])
            end

            #   mise à jour de la fontion objectif
            column_cost = column["cost"]
            set_objective_function(mastermodel, objective_function(mastermodel) + column_cost*z[end])

            # Affichage du cycle ajouté
            cycle = findall(y -> y!=0, column["vertices"])
            push!(cycle, cycle[1])
            # println(" \e[93m|\e[00m  Ajout du cycle $cycle \e[00m")
        end
        
        
        # Si une meilleur borne supérieure a été trouvé lors de l'ajout de la colonne,
        # alors on met à jour la borne supérieure de ce noeud
        if nodeub <= tree[nodeindex].ub
            tree[nodeindex].ub = nodeub
        end
        
        
        # CONVERGENCE DE L'ALGORITHME
        # quand l'algorithme a convergé (écart relatif entre les bornes épslionesque ou coût réduit négatif ou nul)
        if 2*abs((tree[nodeindex].ub - nodelb))/abs((tree[nodeindex].ub + nodelb)) < ϵ || SPobj <= 0
            println(" \e[93m|\e[00m  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : $nodelb")
            println(" \e[93m|\e[00m                                               et la borne supérieure : $(tree[nodeindex].ub)")
            
            # Si la colonne artificielle est utilisée dans la solution,       ##################### Revoir
            # alors ce noeud est irréalisable
            if solution[KEEP_INIT_COLUMNS] >= ϵ
                println(" \e[93m|\e[31m  Ce noeud est irréalisable et donc élagué (la colonne artificielle est encore présente)\e[00m")
                return []
            end

            # Sinon, on retourne la solution de ce noeud à l'aide de la fonction 'calculate_xsol',
            # qui renvoie les variables x_ij des arcs   
            return calculate_xsol(solution, node_pool)
        end
    end
end